# BRCA Subtype Prediction Examination Script


-----------------------------------------
Instructions:

This exam assesses your knowledge of supervised learning and model evaluation.
You are required to predict BRCA subtypes using gene expression data filtered to PAM50 genes.
Complete all tasks below and ensure that your code is well-commented and reproducible.

# Preloaded Packages (Do NOT modify this section)

In [1]:
# load the required packages 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Section 0: Load the Required Data [0 marks - Already provided]

In [2]:
# Step 1: Load the clinical information
clinical_file = 'brca_tcga_data_clinical_patient.txt'
clinical_df = pd.read_csv(clinical_file, sep='\t', skiprows=4)
clinical_df.head()

# Step 2: Load the mRNA expression (median values)
mrna_file = 'brca_tcga_data_RNA_Seq_v2_expression_median.txt'
mrna_df = pd.read_csv(mrna_file, sep='\t')

# Remove the Entrez_Gene_Id column if it exists
if 'Entrez_Gene_Id' in mrna_df.columns:
    mrna_df.drop(columns=['Entrez_Gene_Id'], inplace=True)

# Step 3: Load the PAM50 genes
pam50_df = pd.read_csv('PAM50_genes.txt')

# Step 4: Filter mRNA data to include only PAM50 genes
mrna_df = mrna_df[mrna_df['Hugo_Symbol'].isin(pam50_df['Hugo_Symbol'])]

# Step 5: Reshape mRNA data so that each row is a patient and each column is a gene
mrna_long = mrna_df.set_index('Hugo_Symbol').T.reset_index()
mrna_long = mrna_long.rename(columns={'index': 'PATIENT_ID'})

# Step 6: Clean PATIENT_IDs for consistency
mrna_long['PATIENT_ID'] = mrna_long['PATIENT_ID'].str.replace('_', '-', regex=False).str[:12]
clinical_df['PATIENT_ID'] = clinical_df['PATIENT_ID'].str.replace('_', '-', regex=False)

# Step 7: Merge clinical data with mRNA expression data
clinical_subset = clinical_df[['PATIENT_ID', 'SUBTYPE']]
data = pd.merge(clinical_subset, mrna_long, on='PATIENT_ID', how='inner')

# Step 8: Remove samples with NA subtype and clean labels
data = data[data['SUBTYPE'] != 'NA']
data['SUBTYPE'] = data['SUBTYPE'].str.replace('_', '-', regex=False)
data['SUBTYPE'] = data['SUBTYPE'].astype('category')

# Display the shapes of the datasets to understand their dimensions
print("This is is the size of the data :", data.shape)  # Shape of the training data

# check the number of samples in each category 
print("\nNumber of samples if each subtype category :", data['SUBTYPE'].value_counts())

# the size of the data 
print("\nHere is the header of the data:")
data.head()


This is is the size of the data : (1082, 49)

Number of samples if each subtype category : SUBTYPE
BRCA-LumA      499
BRCA-LumB      197
BRCA-Basal     171
BRCA-Her2       78
BRCA-Normal     36
Name: count, dtype: int64

Here is the header of the data:


,PATIENT_ID,SUBTYPE,ACTR3B,ANLN,BAG1,BCL2,BLVRA,CCNB1,CCNE1,CDC20,...,PHGDH,PTTG1,RRM2,SFRP1,SLC39A6,TMEM45B,TYMS,TYRP1,UBE2C,UBE2T
0,TCGA-3C-AAAU,BRCA-LumA,359.824,828.215,1890.570,2315.760,850.618,1282.820,163.3680,603.153,...,741.361,383.605,1477.210,473.2170,90765.90,2.4126,493.114,1.0340,555.590,318.465
1,TCGA-3C-AALI,BRCA-Her2,144.644,1566.070,728.026,264.818,1873.300,1877.650,227.8410,648.722,...,759.652,553.018,3219.140,101.6860,1699.29,799.3470,719.320,116.3680,1351.280,1004.350
2,TCGA-3C-AALJ,BRCA-LumB,153.219,637.353,1869.720,2538.530,2153.220,1822.300,178.6040,873.980,...,106.981,802.357,1305.530,67.0898,15816.90,5.4397,667.017,131.4600,1195.830,757.026
3,TCGA-3C-AALK,BRCA-LumA,135.292,631.775,1651.160,2172.110,1327.270,827.058,44.2698,539.098,...,694.663,311.957,909.392,1236.6600,19032.70,532.4780,724.692,2.0687,496.897,373.190
4,TCGA-4H-AAAK,BRCA-LumA,235.319,359.575,2154.500,1575.320,651.064,647.660,56.5957,389.362,...,626.383,257.872,384.255,1531.9100,3834.89,385.1060,675.723,0.8511,322.128,192.340


# Section 1: Prepare the Dataset [6 marks]

In [4]:
# Task:
# Q1. Extract gene expression features and subtype labels from the merged dataset. [1 mark]
# Q2. Standardize the gene expression features using Z-score normalization. [2 marks]
# Q3. Split the dataset into training and test sets using a 70/30 stratified split. [2 mark]
# Q4. Report the number of instances in each class in both training and test sets. [1 mark]

# ****** write your code here *******
# Q1. Extract gene expression features and subtype labels from the merged dataset.
X = data.drop(columns=['PATIENT_ID', 'SUBTYPE'])  # Gene expression features
y = data['SUBTYPE']  # Subtype labels

# Check for NaN in target variable and remove those rows
nan_mask = y.isna()
if nan_mask.any():
    print(f"Warning: Found {nan_mask.sum()} NaN values in target variable, removing these rows")
    X = X[~nan_mask]
    y = y[~nan_mask]

# Q2. Standardize the gene expression features using Z-score normalization.
# Also handle potential NaN in features by filling with mean
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')  # Fill NaN with mean of the column
scaler = StandardScaler()

X_imputed = imputer.fit_transform(X)  # First handle missing values
X_scaled = scaler.fit_transform(X_imputed)  # Then standardize

# Q3. Split the dataset into training and test sets using a 70/30 stratified split.
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, 
                                                    stratify=y, random_state=42)

# Q4. Report the number of instances in each class in both training and test sets.
print("\nTraining set class distribution:")
print(y_train.value_counts())
print("\nTest set class distribution:")
print(y_test.value_counts())
print(f"\nTraining set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")


# ***********************************

# Write you comment here on the number of instances in each class and the size of the training / test set: 
# Your explanation here: Training set size: 686; Test set size: 295


Training set class distribution:
SUBTYPE
BRCA-LumA      349
BRCA-LumB      138
BRCA-Basal     120
BRCA-Her2       54
BRCA-Normal     25
Name: count, dtype: int64

Test set class distribution:
SUBTYPE
BRCA-LumA      150
BRCA-LumB       59
BRCA-Basal      51
BRCA-Her2       24
BRCA-Normal     11
Name: count, dtype: int64

Training set size: 686
Test set size: 295


# Section 2: Train Statistical Learning Models [8 marks]

In [5]:
# Task:
# - Train three different classifiers (e.g., Logistic Regression, Random Forest, SVM). [3 marks]
# - Evaluate each using accuracy, confusion matrix, classification report, and AUC. [3 marks]
# - Summarize and compare the performance. [2 marks]

# ****** write your code here *******
# Initialize classifiers
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "SVM": SVC(probability=True, random_state=42)  # probability=True for AUC calculation
}

# Dictionary to store results
results = {}

# Train and evaluate each model
for name, model in models.items():
    print(f"\n{'='*50}")
    print(f"Training and evaluating {name}")
    print(f"{'='*50}")
    
    # Train model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test) if hasattr(model, "predict_proba") else None
    
    # Calculate metrics
    accuracy = model.score(X_test, y_test)
    print(f"\nAccuracy: {accuracy:.4f}")
    
    # Confusion matrix
    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # AUC (One-vs-Rest for multiclass)
    if y_proba is not None:
        auc_score = roc_auc_score(y_test, y_proba, multi_class='ovr')
        print(f"AUC (OvR): {auc_score:.4f}")
    else:
        auc_score = None
        print("AUC not available (model doesn't support probability estimates)")
    
    # Store results
    results[name] = {
        'accuracy': accuracy,
        'confusion_matrix': cm,
        'classification_report': classification_report(y_test, y_pred, output_dict=True),
        'auc': auc_score
    }

# Compare model performance
print("\n\nModel Comparison:")
print("{:<20} {:<10} {:<10}".format("Model", "Accuracy", "AUC"))
print("-"*40)
for name, res in results.items():
    print("{:<20} {:<10.4f} {:<10.4f}".format(
        name, 
        res['accuracy'], 
        res['auc'] if res['auc'] is not None else float('nan')
    ))



# ***********************************

# Your comment here on the performance of the models:Random Forest performed best (89.2% accuracy, 0.988 AUC) but struggled with rare subtypes. Logistic Regression balanced accuracy (87.1%) and interpretability, while SVM trailed (82.7%) but maintained decent AUC (0.965). 
#All excelled at identifying BRCA-Basal (98-100% recall) but performed poorly on BRCA-Normal. Random Forest is ideal for clinical use, Logistic Regression for interpretable research, and SVM may improve with tuning. 



Training and evaluating Logistic Regression

Accuracy: 0.8712

Confusion Matrix:
[[ 50   1   0   0   0]
 [  0  20   2   2   0]
 [  0   2 141   7   0]
 [  0   2  13  43   1]
 [  1   0   7   0   3]]

Classification Report:
              precision    recall  f1-score   support

  BRCA-Basal       0.98      0.98      0.98        51
   BRCA-Her2       0.80      0.83      0.82        24
   BRCA-LumA       0.87      0.94      0.90       150
   BRCA-LumB       0.83      0.73      0.77        59
 BRCA-Normal       0.75      0.27      0.40        11

    accuracy                           0.87       295
   macro avg       0.84      0.75      0.77       295
weighted avg       0.87      0.87      0.86       295

AUC (OvR): 0.9774

Training and evaluating Random Forest

Accuracy: 0.8915

Confusion Matrix:
[[ 51   0   0   0   0]
 [  0  20   3   1   0]
 [  0   0 147   3   0]
 [  0   0  16  43   0]
 [  0   0   9   0   2]]

Classification Report:
              precision    recall  f1-score   support



# Section 3: Train Neural Network Models [16 marks]

In [8]:
# Task:
# Q8. Design and train three different neural network architectures: Shallow, Medium, and Deep. [6 marks]
# Q9. Evaluate the models on the test set using classification report, confusion matrix, and AUC. [6 marks]
# Q10. Compare model performances and discuss the observed trends. [4 marks]

# ****** write your code here *******
from tensorflow.keras.layers import BatchNormalization

# First convert string labels to numerical indices
from sklearn.preprocessing import LabelEncoder

# Create label encoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Now convert to one-hot encoding
y_train_oh = to_categorical(y_train_encoded)
y_test_oh = to_categorical(y_test_encoded)

# Define architectures
models = {
    "Shallow": Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.2),
        Dense(y_train_oh.shape[1], activation='softmax')
    ]),
    
    "Medium": Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(y_train_oh.shape[1], activation='softmax')
    ]),
    
    "Deep": Sequential([
        Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
        BatchNormalization(),
        Dropout(0.4),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(y_train_oh.shape[1], activation='softmax')
    ])
}

# Train and evaluate models
results = {}
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

for name, model in models.items():
    print(f"\n{'='*50}")
    print(f"Training {name} Network")
    print(f"{'='*50}")
    
    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    history = model.fit(X_train, y_train_oh,
                       epochs=100,
                       batch_size=32,
                       validation_split=0.2,
                       callbacks=[early_stop],
                       verbose=1)
    
    # Evaluate
    y_pred = model.predict(X_test)
    y_pred_class = np.argmax(y_pred, axis=1)
    
    # Convert back to original labels for reporting
    y_pred_labels = label_encoder.inverse_transform(y_pred_class)
    y_test_labels = label_encoder.inverse_transform(y_test_encoded)
    
    print("\nClassification Report:")
    print(classification_report(y_test_labels, y_pred_labels))
    
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test_labels, y_pred_labels))
    
    auc = roc_auc_score(y_test_oh, y_pred, multi_class='ovr')
    print(f"\nAUC: {auc:.4f}")
    
    results[name] = {
        'classification_report': classification_report(y_test_labels, y_pred_labels, output_dict=True),
        'confusion_matrix': confusion_matrix(y_test_labels, y_pred_labels),
        'auc': auc,
        'history': history
    }

# Compare performance
print("\nModel Comparison:")
print("{:<10} {:<10} {:<10}".format("Model", "Accuracy", "AUC"))
print("-"*30)
for name, res in results.items():
    acc = res['classification_report']['accuracy']
    print("{:<10} {:<10.4f} {:<10.4f}".format(name, acc, res['auc']))


# ***********************************
# Your explanation here: The Medium network achieved the highest accuracy (85.76%), slightly outperforming the Shallow (85.42%) and Deep (84.07%) networks, indicating that deeper architectures may yield diminishing returns for this task. 
    #In terms of training dynamics, the Shallow network trained most stably with minimal divergence between training and validation accuracy, while the Deep network displayed more volatile validation trends. Overfitting was evident in all models, with the Medium network managing it best due to regularization techniques like batch normalization and dropout.
    #All models struggled with the rare BRCA-Normal class, but performed well on BRCA-Basal, with the Medium network achieving the most balanced classification. Finally, AUC analysis confirmed the Medium network’s superior discriminative ability (0.9756), although all models showed comparable ranking performance with minor AUC differences.

C:\Users\ronal\py312_env\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Training Shallow Network
Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 182ms/step - accuracy: 0.4252 - loss: 1.3644 - val_accuracy: 0.6232 - val_loss: 1.0631
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - accuracy: 0.7251 - loss: 0.8500 - val_accuracy: 0.6884 - val_loss: 0.8533
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.7671 - loss: 0.6934 - val_accuracy: 0.7391 - val_loss: 0.7299
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.8030 - loss: 0.5757 - val_accuracy: 0.7754 - val_loss: 0.6564
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - accuracy: 0.8121 - loss: 0.5130 - val_accuracy: 0.8043 - val_loss: 0.5978
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 141ms/step - accuracy: 0.8414 - loss: 0.4752 - val_accuracy: 0.8043 - val_loss: 0.5617
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - accuracy: 0.8519 - loss: 0.4192 - val_accuracy: 0.8116 - val_loss: 0.5388
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - accuracy: 0.8549 - lo

# Final Reflections [10 marks]

In [ ]:
# Q11. Comment on the overall performance of the models trained (statistical and neural networks). [4 marks]
# Q12. How did class imbalance affect model training and evaluation metrics? [3 marks]
# Q13. Why might certain BRCA subtypes be harder to classify accurately than others? [3 marks]

# ****** write your explanation here *******
# Q11: All models performed well (85-89% accuracy), with Random Forest and Medium NN performing best. 
#      Neural networks showed more overfitting than statistical models. Performance was strongest 
#      for common subtypes (Basal, LumA) across all models.

# Q12: Class imbalance hurt recall for rare subtypes (Normal: <55%) despite high precision. 
#      Statistical models handled imbalance slightly better than NNs. F1-scores revealed this 
#      weakness better than accuracy alone.

# Q13: Harder subtypes (Normal, Her2) likely have: 1) Fewer samples, 2) Overlapping gene expression 
#      patterns, and/or 3) Higher biological variability. LumB also confused models due to 
#      intermediate features with LumA.